In [ ]:
#importing all the packages 
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.size': 22})

import pandas as pd
from sklearn import datasets, linear_model
from sklearn.datasets import load_iris
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

### Loading and preprocessing the dataset

In [ ]:
# we will use the handwritten MNIST digit dataset

digits_mnist = tf.keras.datasets.mnist.load_data()

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = digits_mnist

In [ ]:
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train.shape


In [ ]:
X_train.dtype

In [ ]:
# scaling the pixel intensities
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.


### Defining the model Architecture 

In [ ]:
# defining the model (one way to do it) 
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
# same as above

#model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=[28, 28]),
    #tf.keras.layers.Dense(300, activation="relu"),
    #tf.keras.layers.Dense(100, activation="relu"),
    #tf.keras.layers.Dense(10, activation="softmax")
#])

### Displaying the model Architecture 

In [ ]:
model.summary()

### Compiling the model 

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

### Saving and loading the model

In [ ]:
model.save("keras_digits_model", save_format="tf")


In [ ]:
# extra code – show the contents of the my_keras_model/ directory
from pathlib import Path
for path in sorted(Path("keras_digits_model").glob("**/*")):
    print(path)

In [ ]:
model_load  = tf.keras.models.load_model("keras_digits_model")

### Model Callbacks

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_mnist_model", save_best_only=True)
#history = model.fit(X_train, y_train, epochs=30,verbose =0, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=30,verbose =0, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])


In [ ]:
model = tf.keras.models.load_model("my_mnist_model") # rollback to best model
model.evaluate(X_test, y_test)

### Adding Batch Normalization 

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

### Adding BN layer before activation 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

### Learning Rate Scheduler 

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)


### layer with l2 regularization

In [ ]:
layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

### Model with dropout

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])